# Schneider Electrics - Energy Consumption Forecast

**End of competition: 31st March 23:59**

*In the context of this challenge, we do not want to look at all of the details of the building--the objective of the challenge is to provide an algorithm that can*
- *(i) either make a good forecast for all or some of the buildings*

*or* 
- *(ii) bring the conclusion that other data would be necessary to make relevant forecasts.*

https://www.drivendata.org/competitions/51/electricity-prediction-machine-learning/page/101/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading of the datasets & EDA
https://www.drivendata.org/competitions/51/electricity-prediction-machine-learning/data/


### Metadata of the buidlings (metadata.csv)
- *SiteId*: ID of the building (range from 1 to 305 with missing values)
- *Binary columns*: xx_IsDayOff from monday to sunday (regular days-off of the building)
- *Numeric columns*: Surface, Sampling (number of minutes between each observation for this site), BaseTemperature (of the building in °C)

In [ ]:
#Loading the file
metadata = pd.read_csv('metadata.csv')
metadata['Sampling'] = metadata['Sampling'].astype('category')
#Turning xxIsDayOff in dummy variables
for lab in metadata.iloc[:, 4:].columns:
    metadata[lab] = metadata[lab].map({False: 0, True:1})
#Total days-off per week as categorical type
metadata['Total_DaysOff'] = metadata.iloc[:, 4:].sum(axis=1).astype('category')

print(metadata.head())
print(metadata.tail())
print(metadata.info())

In [ ]:
#Graphic visualizations of numerical values
_ = metadata.plot(kind='scatter', x='SiteId', y='SiteId', figsize=(10,5))
plt.show()

_ = metadata.plot(kind='scatter', x='SiteId', y='Surface', figsize=(10,5))
plt.show()

_ = metadata.plot(kind='box', y='Surface', figsize=(5,10))
plt.show()

_ = metadata.plot(kind='scatter', x='SiteId', y='BaseTemperature', figsize=(10,5))
plt.show()

_ = metadata.plot(kind='bar', x='SiteId', y='Total_DaysOff', figsize=(10,5))
plt.show()


### Public Holidays (holidays.csv)
Public holidays at the sites included in the dataset, which may be helpful for identifying days where consumption may be lower than expected.

In [ ]:
#Loading the file
holidays = pd.read_csv('holidays.csv', parse_dates=[1])
holidays.index = holidays['Date']
holidays = holidays.drop(['Unnamed: 0'], axis='columns')
holidays['SiteId'] = holidays['SiteId'].astype('category')
print(holidays.head())
print(holidays.tail())
print(holidays.info())

#Days off count per building
holidays_count = holidays.groupby('SiteId').count()

_ = holidays_count['Holiday'].plot(kind='bar', figsize=(20, 5))
plt.show()

### Historical Consumption - Training set (train.csv)


- *obs_id*: Arbitrary ID for the observation
- *SiteId*: ID number for the building that matches across datasets
- *ForecastId*: ID for a timeseries that is part of a forecast (can be matched with the submission file)
- *Timestamp*: The time of the measurement
- *Value*: A measure of consumption for that building


In [ ]:
#Loading the file
training = pd.read_csv('train.csv', parse_dates=[2])
training.index = training['Timestamp']
#training = training.drop(['Timestamp'], axis=1)
training['SiteId'] = training['SiteId'].astype('category')

print(training.head())
print(training.tail())
print(training.info())

In [ ]:
#NaN values = 1.3% of the "Value" column
print(len(training[training['Value'].isnull()]) / len(training))

#Visualization of means for each building
site_means = training.groupby('SiteId')['Value'].mean()
_ = site_means.plot(kind='bar', figsize=(30,20))
plt.show()

In [ ]:
#Evolution of power consumption for 1 building
one_building = training[training['SiteId'] == 1][['Value', 'Timestamp']]
one_building['week_day'] = one_building['Timestamp'].dt.dayofweek.astype('category')
one_building['month'] = one_building['Timestamp'].dt.month.astype('category')
#one_building['day_year'] = one_building['Timestamp'].dt.dayofyear.astype('category')
print(one_building.head(10))

#Mean according to day in the week
day_mean = one_building.groupby('week_day')['Value'].mean()
print(day_mean)

#Mean according to month of the year
month_mean = one_building.groupby('month')['Value'].mean()
print(month_mean)

#Power evolution
_ = one_building.plot(y='Value', figsize=(20,5))
plt.show()

#Days-off for the SiteID 1 are Saturday and sunday
_ = one_building['2014-09-04' : '2014-09-12'].plot(y='Value')
plt.show()

#Means
_ = day_mean.plot()
plt.show()
_ = month_mean.plot()
plt.show()


- *For a buidling, the consumption seems to be higher in May-June-july (air conditionning ?)*
- *The power consumption substantially drops during days-off of the building*

### Weather data (weather.csv)
temperature data from several stations near each site. For each site several temperature measurements were retrieved from stations in a radius of 30 km if available

In [ ]:
weather = pd.read_csv('weather.csv', index_col=0, parse_dates=[1])
weather.index = weather['Timestamp']
weather = weather.drop(['Timestamp'], axis=1)
weather['SiteId'] = weather['SiteId'].astype('category')

print(weather.head(30))
print(weather.info())

In [ ]:
#Temperature means per site
mean_t_sites = weather.drop(columns=['Distance'])
mean_t_sites = mean_t_sites.groupby('SiteId').mean()
print(mean_t_sites.head(10))

#Visualization
_ = mean_t_sites.plot(kind='bar', figsize=(20,10))
plt.show()

*--> Highly depends on the number of temperature records !*

*--> No assumption that all the buildings are in the same city*

### Submission format  
The value is the target variable, for the given timestamp & site

Same structure as the training dataset, useful to preprocess the submission csv

In [ ]:
forecast = pd.read_csv('submission_format.csv', parse_dates=[2])
print(forecast.head())
print(forecast.tail())
print(forecast.info())

building1 = forecast[forecast['SiteId'] == 1]
print("\n Site 1 \n", building1.head())
print(building1.tail())
building5 = forecast[forecast['SiteId'] == 5]
print("\n Site 5 \n", building5.head())
print(building5.tail())

---

## Full Training set preparation

- Missing values for meaurements: fill with value in-between ? Drop as it's target variable?


**FEATURES**: 
- Size of the flat, 
- dya of week and month of year (extracted from timestamp : https://pandas.pydata.org/pandas-docs/stable/api.html#datetimelike-properties ),
- regular days-off, 
- national holidays, 


### Building of the full training set
#### Preprocessing of the training dataset
Saved under *'training_features_DB.csv'*

In [ ]:
#All functions combined to get the full training dataset 
#besides temperatures at this point

def add_features(file):
    """Adds to the train.csv or the submission_format.csv the corresponding features
    except the temperature records"""
    
    
    #Loading the original training dataset
    training_preprocess = pd.read_csv(file, parse_dates=[2])
    training_preprocess.index = training_preprocess['Timestamp']
    training_preprocess['SiteId'] = training_preprocess['SiteId'].astype('category')
    #print(training_preprocess.head())
    #print(training_preprocess.info())


    #Drop NaN values for the target variable
    training_preprocess = training_preprocess.loc[training_preprocess['Value'].notnull()]


    #Add 2 columns containing respectively the day of the week [0;6] and the month number [1;12] as features
    training_preprocess['Day_Week'] = training_preprocess['Timestamp'].dt.dayofweek.astype('category')
    training_preprocess['Month'] = training_preprocess['Timestamp'].dt.month.astype('category')


    #Add national holidays as a feature according to the timestamps and the building Id    
    #Loading holidays file
    holidays = pd.read_csv('holidays.csv', parse_dates=[1])
    holidays.index = holidays['Date']
    holidays = holidays.drop(['Unnamed: 0'], axis='columns')
    holidays.columns.values[0] = 'Timestamp'
    holidays.columns.values[1] = 'National_Holiday'

    #Merging the dataframes to add the corresponding National holidays of the building
    training_preprocess = pd.merge(training_preprocess, holidays,
                                   how='left', on=['Timestamp','SiteId'])


    #Replace NaN by 0 and values by 1 to have a dummy variable
    training_preprocess.loc[training_preprocess['National_Holiday'].notnull(), 'National_Holiday'] = 1
    training_preprocess['National_Holiday'] = training_preprocess['National_Holiday'].fillna(0)


    #Get dummy features
    training_preprocess = pd.get_dummies(training_preprocess)


    #Add surface, sampling frequency(min) & base temperature of the building according to SiteId from metadata
    #Loading metadata
    metadata = pd.read_csv('metadata.csv')

    #Merging dataframes
    training_preprocess = pd.merge(training_preprocess, 
                                   metadata[['SiteId', 'Surface', 'Sampling', 'BaseTemperature']],
                                   how='left', on=['SiteId'])

    
    print(training_preprocess.head(20))
    print(training_preprocess.info())
    
    training_preprocess.to_csv('{}_features_DB.csv'.format(file[:-4]))
    
    return training_preprocess

In [ ]:
#Add features to the training set
add_features('train.csv')

*The Sampling column is not a feature but an help to get the correct temperature up/down sampling*

In [ ]:
#Add features to the submission csv
add_features('submission_format.csv')

#### Site with most records

In [ ]:
#Count number of records per site to pick up the biggest one for machine learning
records_count = training_preprocess.groupby('SiteId').count()['Value'].sort_values(ascending=False)

_ = records_count.plot(kind='bar', figsize=(20,5))
plt.show()

print(records_count.head())

*SiteId = 162 has 53 687 records, enough to train the ML model*

### Visualization of the features vs consumption for 1 building

- Difference between regular day-off and national holiday, especially if the heating/air conditionning is scheduled and do not take into account national holidays
- The feature is not temperature but the difference betwen base temp and outdoor temp (absolute value?), containing 2 information instead of one, and probably trigerring the heating/air conditionning system

In [ ]:
training_preprocess = pd.read_csv('train_features_DB.csv', index_col=0, parse_dates=[3])
training_preprocess['SiteId'] = training_preprocess['SiteId'].astype('int')
print(training_preprocess.info())


def building_features(SiteId, viz=False):
    """For he specified building, creates a training set with the features from
    the weather and metadata csv files
    If vis=True, enables plotting of consumption means and temperature evolution"""

    dataset = training_preprocess.loc[training_preprocess['SiteId'] == SiteId]
    dataset.index = dataset['Timestamp']
    #print(dataset.head())
    #print(dataset.info())

    if viz == True:
        #Evolution consumption
        _ = dataset.plot(y='Value', figsize=(20,10), 
                         title="Consumption records of the building {}".format(SiteId))
        plt.show()

        #Subplots of consumption on all days of the week
        means = dict()
        for i in range(7):
            day = 'Day_Week_' + str(i)
            means[day] = dataset[dataset[day] == 1]['Value'].mean()
        means['National_holiday'] = dataset[dataset['National_Holiday'] == 1]['Value'].mean()
        _ = pd.Series(means).plot(kind='bar', figsize=(20,10),
                                 title="Consumption mean for days of week for building {}".format(SiteId))
        plt.show()

        #Subplots of consumption on months
        means = dict()
        for i in range(1,13):
            month = 'Month_' + str(i)
            means[month] = dataset[dataset[month] == 1]['Value'].mean()
        _ = pd.Series(means).plot(kind='bar', figsize=(20,10),
                                 title="Consumption mean for months for building {}".format(SiteId))
        plt.show()
        

    #Diff Temp vs consumption
    #Get temperatue with the righ sampling
    weather = pd.read_csv('weather.csv', index_col=0, parse_dates=[1])
    weather = weather.loc[weather['SiteId'] == SiteId]
    weather.index = weather['Timestamp']
    weather = weather.drop(['Distance'], axis=1)
    weather = weather.drop(['Timestamp', 'SiteId'], axis=1)

    if viz == True:
        #Plot temperatures
        _ = weather.plot(figsize=(20,10), 
                         title="Temperature records for building {}".format(SiteId))
        plt.show()
        
    #Resample the temperatures to match the energy consumption in the training set
    SAMP = str(int(dataset['Sampling'].values[0])) + 'min'
    weather_resampled = weather['Temperature'].resample(SAMP).first().interpolate('linear')
    #print(weather_resampled.head())
    #print(dataset.head())

    #Merge the resampled temperature to the training set
    dataset = dataset.join(weather_resampled)

    #Get the difference from the Base Temperature
    BASET = dataset['BaseTemperature'].values[0]
    dataset['Diff_Temp'] = dataset['Temperature'] - BASET

    #Add the hour of day feature
    dataset['Hour_Day'] = dataset['Timestamp'].dt.hour.astype('category')
    dataset = pd.get_dummies(dataset)

    #Drop non feature columns from the training set
    dataset = dataset.drop(columns=['BaseTemperature', 'Temperature', 'Sampling','Surface', 
                                    'ForecastId','Timestamp', 'SiteId', 'obs_id'])

    #Drop extra dummy features (1 in day, 1 in month & 1 in hours of day)
    dataset = dataset.drop(columns=['Day_Week_0', 'Month_1', 'Hour_Day_0'])

    dataset.to_csv('training_building{}_DB.csv'.format(SiteId))

    #print(dataset.head(20))
    
    return dataset

In [ ]:
building_features(162)

*--> The consumption is reduced on sundays and national holidays compared to weekdays and saturday (even if holiday too)*


In [ ]:
building_features(1)

In [ ]:
building_features(302)

#### Create the csv training sets for all buildings

In [ ]:
SITES = training_preprocess['SiteId'].unique().tolist()
#print(SITES)

for site in SITES:
    print("Building {}".format(site))
    building_features(site)

---

## Machine Learning models
Target: energy consumption forecast for each building

- Preprocessing: 
    - getting data from metadata globally
    - getting temperatures for one given building
    - dummy variable for site ID, Imputing, Scaling

- Hashing for computing efifciency

- Regression Models: Ridge, Lasso, ElasticNet

- Hyperparameter tuning with gridsearch

### SImple Model for 1 building
- Training set: see above for 1 building
- Features: day of week, month, dayoff, national holiday,  /\temperature
- Preprocessing: scaling (temperature), imputing?, dummy variables (already done)
- Model: Simple Ridge regression (hyperparameter = alpha)

*Test with the building 162*

#### Function for preprocessing/addind features
- Training & Submission sets: only Obs_Id, Timestamp, SiteId, ForecastId, Value
- Features to add:
    - Temperature: 
    - Days off (regular & bank-holidays)
    - Day of month, hour of day
    
Creates function to go into the pieline using FunctionTransformer()

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, ElasticNet, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler, FunctionTransformer

In [ ]:
%%time

SiteId = 162

#Load training set
training = pd.read_csv('training_building{}_DB.csv'.format(SiteId), index_col=0)
print(training.head())
print(training.info())
#print(training.columns)

#Create the features (X) and target (y) arrays
y = training.loc[:,'Value'].values.reshape((-1,1))
X = training.drop(columns=['Value']).values

print('shape of X and Y arrays: {}'.format([X.shape, y.shape]))


#Building the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

#Alpha Hyperparameter tunning
param_grid={'reg__alpha': np.logspace(-4,0,50)}

#Pipeline building
pipeline = Pipeline([('reg', Ridge(normalize=True))
                    ])

cv = GridSearchCV(pipeline, param_grid=param_grid, cv=10)

cv.fit(X_train, y_train)
predictions = cv.predict(X_test)

print(cv.best_params_)
print(cv.best_score_)
print('Score R² on test set: {}'.format(cv.score(X_test, y_test)))

#Simple RMSE for 1 building
mu = 1 / np.mean(y_test)
simple_rmse = mu * np.sqrt(np.sum((predictions - y_test) ** 2))
print('RMSE result: {}'.format(simple_rmse))

- **Best results with Ridge:**
*cv=10  |  normalize=True  |  {'reg__alpha': 0.0001}  |  Best obtained R²: 0.7683974571210929  |  Score R² on test set: 0.77008  |  RMSE = 23.26860


In [ ]:
%%time

SiteId = 1

#Load training set
training = pd.read_csv('training_building{}_DB.csv'.format(SiteId), index_col=0)
print(training.head())
print(training.info())
print(training.columns)

#Create the features (X) and target (y) arrays
y = training.loc[:,'Value'].values.reshape((-1,1))
X = training.drop(columns=['Value']).values

print('shape of X and Y arrays: {}'.format([X.shape, y.shape]))


#Building the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

#Alpha Hyperparameter tunning
param_grid={'reg__alpha': np.logspace(-4,0,50)}

#Pipeline building
pipeline = Pipeline([('reg', Ridge(normalize=True))
                    ])

cv = GridSearchCV(pipeline, param_grid=param_grid, cv=10)

cv.fit(X_train, y_train)
predictions = cv.predict(X_test)

print(cv.best_params_)
print(cv.best_score_)
print('Score R² on test set: {}'.format(cv.score(X_test, y_test)))

#Simple RMSE for 1 building
mu = 1 / np.mean(y_test)
simple_rmse = mu * np.sqrt(np.sum((predictions - y_test) ** 2))
print('RMSE result: {}'.format(simple_rmse))

In [ ]:
%%time

SiteId = 302

#Load training set
training = pd.read_csv('training_building{}_DB.csv'.format(SiteId), index_col=0)
print(training.head())
print(training.info())
print(training.columns)

#Create the features (X) and target (y) arrays
y = training.loc[:,'Value'].values.reshape((-1,1))
X = training.drop(columns=['Value']).values

print('shape of X and Y arrays: {}'.format([X.shape, y.shape]))


#Building the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

#Alpha Hyperparameter tunning
param_grid={'reg__alpha': np.logspace(-4,0,50)}

#Pipeline building
pipeline = Pipeline([('reg', Ridge(normalize=True))
                    ])

cv = GridSearchCV(pipeline, param_grid=param_grid, cv=10)

cv.fit(X_train, y_train)
predictions = cv.predict(X_test)

print(cv.best_params_)
print(cv.best_score_)
print('Score R² on test set: {}'.format(cv.score(X_test, y_test)))

#Simple RMSE for 1 building
mu = 1 / np.mean(y_test)
simple_rmse = mu * np.sqrt(np.sum((predictions - y_test) ** 2))
print('RMSE result: {}'.format(simple_rmse))

### Pipeline for the submission csv, predictions on buildings individually
Pipeline that does predictions for each building individually (rather than using the entire dataset to make predictions)

In [4]:
def building_features_sub(SiteId, dataset):
    """For the specified building in the submission format file (filtered submission), 
    creates a set for predictions with the features from
    the weather (NOT RESAMPLED HERE) and metadata csv files"""
    

    #Diff Temp vs consumption
    #Get temperatue with the righ sampling
    weather = pd.read_csv('weather.csv', index_col=0, parse_dates=[1])
    weather = weather.loc[weather['SiteId'] == SiteId]
    weather.index = weather['Timestamp']
    weather = weather.drop(['Distance'], axis=1)
    weather = weather.drop(['Timestamp', 'SiteId'], axis=1)

    
    #Resample the temperatures to match the energy consumption in the training set
    SAMP = str(int(dataset['Sampling'].values[0])) + 'min'
    weather_resampled = weather['Temperature'].resample('D').mean()#.first().interpolate('linear')
    time_index = dataset.index
    weather_resampled = weather_resampled.reindex(dataset.index)
        
    #print(weather_resampled.head())
    #print(dataset.head())

    #Merge the resampled temperature to the training set
    dataset['Temperature'] = weather_resampled

    #Get the difference from the Base Temperature
    BASET = dataset['BaseTemperature'].values[0]
    dataset['Diff_Temp'] = dataset['Temperature'] - BASET

    #Add the hour of day feature
    dataset['Hour_Day'] = dataset['Timestamp'].dt.hour.astype('category')
    dataset = pd.get_dummies(dataset)

    #Drop non feature columns from the training set
    dataset = dataset.drop(columns=['BaseTemperature', 'Temperature', 'Sampling','Surface', 
                                    'ForecastId','Timestamp', 'SiteId', 'obs_id'])

    #Drop extra dummy features (1 in day, 1 in month & 1 in hours of day)
    dataset = dataset.drop(columns=['Day_Week_0', 'Month_1', 'Hour_Day_0'])

    #dataset.to_csv('training_building{}_DB.csv'.format(SiteId))

    #print(dataset.head(20))
    
    return dataset

In [6]:
%%time

submission = pd.read_csv('submission_format_features_DB.csv', parse_dates=[3], index_col=0)
submission.index = submission['Timestamp']
#print(submission.head())
#print(submission.info())

SUB = ['obs_id', 'SiteId', 'Timestamp', 'ForecastId']

SITES = submission['SiteId'].unique().tolist()[:3]
print("Buildings in the Pipeline: {}".format(SITES))

results_df = pd.DataFrame()
predictions_df = pd.DataFrame()
R2 = np.empty(len(SITES))
RMSE = np.empty(len(SITES))


for i, site in enumerate(SITES):
    site = int(site)
    print("\n Training of model for building {}\n".format(site))
    
    training_set = pd.read_csv('training_building{}_DB.csv'.format(site), index_col=0)
    #print("\n Training set overview\n", training_set.head())
    #print(training_set.info())
  
    #Create the features (X) and target (y) arrays for the training
    X = training_set.drop(columns=['Value']).values
    y = training_set.loc[:,'Value'].values.reshape((-1,1))

    #print('shape of X and Y arrays: {}'.format([X.shape, y.shape]))


    #Building the model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

    #Alpha Hyperparameter tunning
    param_grid={'reg__alpha': np.logspace(-4,0,30)}

    #Pipeline building
    pipeline = Pipeline([('imp', Imputer(strategy='mean', axis=1)),
                         ('reg', Ridge(normalize=True))
                        ])

    cv = GridSearchCV(pipeline, param_grid=param_grid, cv=10)
    
    #Training on the training set of the building
    cv.fit(X_train, y_train)
    pred_training = cv.predict(X_test)
    score = cv.score(X_test, y_test)
    
    print("Hyperparameter fine tunning:", cv.best_params_, cv.best_score_)
    print('Score R² on test set: {}'.format(score))

    #Simple RMSE for 1 building
    mu = 1 / np.mean(y_test)
    simple_rmse = mu * np.sqrt(np.sum((pred_training - y_test) ** 2))
    print('RMSE result of training: {}'.format(simple_rmse))
    
    #Keep the scores (R² & RMSE) in a dataframe
    R2[i] = score
    RMSE[i] = simple_rmse
    
    
    #Prepare the submission set for predictions
    sub_building = submission.loc[submission['SiteId'] == site]
    results_df = sub_building.loc[:, SUB]
    #print(sub_building.head())
    #print(sub_building.shape)

    #Add temperature and building specific features to submission set
    ##HERE##
    sub_building = building_features_sub(site, sub_building)

    #print(sub_building.shape)
    sub_building = sub_building.drop(columns=['Value'])
    #print("Submission table for predictions: \n", sub_building.head())

    X_sub = sub_building.values
    y_sub = cv.predict(X_sub)
    
    #Append the predictions and submission df with the obtained predictions
    results_df['Value'] = y_sub
    predictions_df = predictions_df.append(results_df)
    scores_df = pd.DataFrame({"Building": SITES, "R2": R2, "RMSE": RMSE})
    
print(predictions_df.head())
print(scores_df.head())
predictions_df.to_csv('submission_format_results_DB.csv')
scores_df.to_csv('scores_DB.csv')

#Display the best scores obtained
max_R2 = np.max(R2)
min_rmse = np.min(RMSE)
print("\n Best performances of the model: R²={} and RMSE={}".format(max_R2, min_rmse))

C:\Users\Damien\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Buildings in the Pipeline: [1, 2, 3]

 Training of model for building 1

Hyperparameter fine tunning: {'reg__alpha': 0.14873521072935117} 0.12619865672087166
Score R² on test set: 0.21524455098354686
RMSE result of training: 7.60757674997267


C:\Users\Damien\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A 


 Training of model for building 2

Hyperparameter fine tunning: {'reg__alpha': 0.0003562247890262444} 0.4401874988322144
Score R² on test set: 0.44706458816892003
RMSE result of training: 49.85531717773496


C:\Users\Damien\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A 


 Training of model for building 3

Hyperparameter fine tunning: {'reg__alpha': 0.01610262027560939} 0.7152283860790387
Score R² on test set: 0.7909868619622098
RMSE result of training: 2.0934737231634966


C:\Users\Damien\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Damien\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A 

             obs_id  SiteId  Timestamp  ForecastId         Value
Timestamp                                                       
2015-08-29  1677832       1 2015-08-29           1  3.270972e+06
2015-08-30  5379616       1 2015-08-30           1  3.494990e+06
2015-08-31   496261       1 2015-08-31           1  4.286233e+06
2015-09-01  4567147       1 2015-09-01           1  4.637941e+06
2015-09-02  3684873       1 2015-09-02           1  4.275754e+06
   Building        R2       RMSE
0         1  0.215245   7.607577
1         2  0.447065  49.855317
2         3  0.790987   2.093474
Best performances of the model: R²=0.7909868619622098 and RMSE=2.0934737231634966
Wall time: 6min 18s


### Full Pipeline, predicts on all building altogether
- Load the submission file
- 
    